In [1]:
from PIL import Image, ImageDraw, ImageFont

In [2]:
def map_value(x, in_min, in_max, out_min, out_max):
        return ((x - in_min) * (out_max - out_min) / (in_max - in_min) + out_min)  

In [11]:
class VisualSimplex:
    def __init__(self, node1, node2, node3, color_x, color_y):
        self.x1 = node1.mapped_x
        self.y1 = node1.mapped_y
        self.x2 = node2.mapped_x
        self.y2 = node2.mapped_y
        self.x3 = node3.mapped_x
        self.y3 = node3.mapped_y
        
        self.color_x = color_x
        self.color_y = color_y
    
    def draw_simplex_x(self, drawer):
        coords_array = [(self.x1, self.y1), (self.x2, self.y2), (self.x3, self.y3)]
        drawer.polygon(coords_array, fill = self.color_x, outline = (0, 0, 0))
        
    def draw_simplex_y(self, drawer):
        coords_array = [(self.x1, self.y1), (self.x2, self.y2), (self.x3, self.y3)]
        drawer.polygon(coords_array, fill = self.color_y, outline = (0, 0, 0))

In [12]:
class Node:
    def __init__(self, nx, ny, value_x, value_y):
        self.unmapped_x = nx
        self.unmapped_y = ny
        self.value_x = value_x
        self.value_y = value_y
        
    def map_coords(self, real_x_min, real_x_max, real_y_min, real_y_max, draw_x_min, draw_x_max, draw_y_min, draw_y_max):
        self.mapped_x = int(map_value(self.unmapped_x, real_x_min, real_x_max, draw_x_min, draw_x_max))
        self.mapped_y = int(map_value(self.unmapped_y, real_y_min, real_y_max, draw_y_min, draw_y_max))

In [383]:
class PostProcessor:
    def __init__(self, bg_color, low_color, high_color):
        self.canv = Image.new('RGB', (900, 900), bg_color)
        self.draw = ImageDraw.Draw(self.canv)
        
        self.simplexes = []
        
        self.nodes = {}
        self.elem_nodes = {}
        
        self.min_x = 1e9
        self.min_y = 1e9
        self.max_x = -1e9
        self.max_y = -1e9
        
        self.min_value_x = 1e9
        self.min_value_y = 1e9
        self.max_value_x = -1e9
        self.max_value_y = -1e9
        
        # Pixels margin on canvas
        self.margin_x = None
        self.margin_y = None
        
        # Color gradient
        self.low_color = low_color
        self.high_color = high_color
        
    def update_borders(self, nx, ny, val_x, val_y):
        if (nx > self.max_x):
            self.max_x = nx
            
        if (nx < self.min_x):
            self.min_x = nx
            
        if (ny > self.max_y):
            self.max_y = ny
            
        if (ny < self.min_y):
            self.min_y = ny
            
            
        if (val_x > self.max_value_x):
            self.max_value_x = val_x
            
        if (val_x < self.min_value_x):
            self.min_value_x = val_x
            
        if (val_y > self.max_value_y):
            self.max_value_y = val_y
            
        if (val_y < self.min_value_y):
            self.min_value_y = val_y
        
    def parse_u_file(self, file_path):
        input_file = open(file_path, 'r')
    
        # Reading nodes data
        n_nodes = int(input_file.readline())
        for i in range(n_nodes):
            data = input_file.readline().split(' ')
            node_id = int(data[0])
            u_x = float(data[1])
            u_y = float(data[2])
            node_x = float(data[3])
            node_y = float(data[4])
            
            new_node = Node(node_x, node_y, u_x, u_y)
            self.nodes[node_id] = new_node
            
            self.update_borders(node_x, node_y, abs(u_x), abs(u_y))
            
        # Reading elems data
        n_elems = int(input_file.readline())
        for i in range(n_elems):
            elem_id = i+1
            elem_nodes = list(map(int, input_file.readline().split(' ')[1:]))
            self.elem_nodes[elem_id] = elem_nodes
            
        input_file.close()
        
    def calc_margins(self):
        x_range = self.max_x - self.min_x
        y_range = self.max_y - self.min_y
        
        if (x_range >= y_range):
            self.margin_x = 10 # Pix
            self.margin_y = 10 + int((890 * (1 - y_range/x_range)) / 2)
        else:
            self.margin_y = 10 # Pix
            self.margin_x = 10 + int((890 * (1 - x_range/y_range)) / 2)
            
    def map_colors(self, value_x, value_y):
        min_R, min_G, min_B = self.low_color
        max_R, max_G, max_B = self.high_color
        
        color_x_R = int(map_value(value_x, self.min_value_x, self.max_value_x, min_R, max_R))
        color_x_G = int(map_value(value_x, self.min_value_x, self.max_value_x, min_G, max_G))
        color_x_B = int(map_value(value_x, self.min_value_x, self.max_value_x, min_B, max_B))
        
        color_y_R = int(map_value(value_y, self.min_value_y, self.max_value_y, min_R, max_R))
        color_y_G = int(map_value(value_y, self.min_value_y, self.max_value_y, min_G, max_G))
        color_y_B = int(map_value(value_y, self.min_value_y, self.max_value_y, min_B, max_B))
        
        color_x = (color_x_R, color_x_G, color_x_B)
        color_y = (color_y_R, color_y_G, color_y_B)
        
        return color_x, color_y
        
        
    def form_simplexes(self):
        
        for elem_key in list(self.elem_nodes.keys()):
            node_id_1, node_id_2, node_id_3 = tuple(self.elem_nodes[elem_key])
            
            self.nodes[node_id_1].map_coords(self.min_x, self.max_x, self.min_y, self.max_y,
                                                     self.margin_x, 900-self.margin_x, self.margin_y, 900-self.margin_y)
            self.nodes[node_id_2].map_coords(self.min_x, self.max_x, self.min_y, self.max_y,
                                                     self.margin_x, 900-self.margin_x, self.margin_y, 900-self.margin_y)
            self.nodes[node_id_3].map_coords(self.min_x, self.max_x, self.min_y, self.max_y,
                                                     self.margin_x, 900-self.margin_x, self.margin_y, 900-self.margin_y)
            
            avg_value_x = abs(self.nodes[node_id_1].value_x + self.nodes[node_id_2].value_x + self.nodes[node_id_3].value_x) / 3
            avg_value_y = abs(self.nodes[node_id_1].value_y + self.nodes[node_id_2].value_y + self.nodes[node_id_3].value_y) / 3
            
            color_x, color_y = self.map_colors(avg_value_x, avg_value_y)
            
            new_simplex = VisualSimplex(self.nodes[node_id_1], self.nodes[node_id_2], self.nodes[node_id_3],
                                       color_x, color_y)
            self.simplexes.append(new_simplex)
            
    def invert_simplexes(self):
        for i in range(len(self.simplexes)):
            self.simplexes[i].y1 = 900 - self.simplexes[i].y1
            self.simplexes[i].y1 = 900 - self.simplexes[i].y1
            self.simplexes[i].y1 = 900 - self.simplexes[i].y1
            
    def draw_mesh(self):
        for simplex in self.simplexes:
            simplex.draw_simplex_y(self.draw)
            
    def draw_legend(self):
        legend_size = 20
        # Low color:
        self.draw.rectangle([(15, 15), (15 + legend_size, 15 + legend_size)], fill = self.low_color, outline = 'blue')
        # High color:
        self.draw.rectangle([(15, 55), (15 + legend_size, 55 + legend_size)], fill = self.high_color, outline = 'blue')
        
        myFont = ImageFont.truetype('font.otf', 13)
        
        self.draw.text((55, 15), f'low = {self.min_value_y:.4e}', (0, 0, 0), font = myFont)
        self.draw.text((55, 55), f'high = {self.max_value_y:.4e}', (0, 0, 0), font = myFont)
        
    def draw_grid(self, grid_step):
        # Grid along oY:
        y_start = 50
        while (y_start < 900):
            self.dotted_hori(y_start)
            self.dotted_vert(y_start)
            y_start += grid_step
            
            
    def draw_centrer(self):
        self.draw.line([(450, 0), (450, 900)], fill = (100, 100, 100))
        self.draw.line([(0, 450), (900, 450)], fill = (100, 100, 100))
        
    def dotted_hori(self, y):
        x_start = 0
        
        while (x_start < 900):
            self.draw.line([(x_start, y), (x_start + 1, y)], fill = (0,0,0))
            x_start += 7
        
    def dotted_vert(self, x):
        y_start = 0
        
        while (y_start < 900):
            self.draw.line([(x, y_start), (x, y_start + 1)], fill = (0,0,0))
            y_start += 7

In [384]:
low_color = (50, 20, 100)
high_color = (200, 240, 240)

pp = PostProcessor((255, 255, 255), low_color, high_color)

In [385]:
pp.parse_u_file('displs_1.txt')

In [386]:
pp.calc_margins()

In [387]:
pp.form_simplexes()

In [388]:
pp.draw_grid(100)
pp.draw_mesh()
pp.draw_legend()

In [389]:
pp.canv.show()